In [2]:
import pandas as pd
import json

In [3]:
with open('coms-clean.json') as json_file:
    coms = json.load(json_file)
with open('evts-detail.json') as json_file:
    evts = json.load(json_file)

In [4]:
dfcom = pd.DataFrame(coms)
dfevt = pd.DataFrame(evts)

### Nombre de communes actuelles

Total

In [5]:
dfcom[dfcom['evtSuppression'].isna()].shape

(37697, 7)

Par type

In [6]:
dfcom[dfcom['evtSuppression'].isna()]['type'].value_counts()

COM     34965
COMD     2215
COMA      517
Name: type, dtype: int64

Aucune commune sans événement de création

In [7]:
dfcom[dfcom['evtCreation'].isna()]

,codeInsee,nom,nomSansArticle,evtCreation,id,type,evtSuppression


Comparaison fichier csv COG datagouv et données sparql

In [8]:
df = pd.read_csv('commune2021.csv')

Filtre hors ARM : arrondissement

In [9]:
listcog = df[df['TYPECOM'] != 'ARM']['COM'].unique().tolist()

Filtre commune qui ont eu une suppression pour ne garder que les actuelles

In [10]:
listrdf = dfcom[dfcom['evtSuppression'].isna()]['codeInsee'].unique().tolist()

Comparaison des deux listes

In [11]:
list(set(listcog) - set(listrdf))

[]

In [12]:
len(listcog)

37077

In [13]:
len(listrdf)

37077

=> On est bon

### Fonctions

Fonctions récupération evt

In [22]:
def getEvtByCodeInsee(codeInsee):
    list_coms = dfcom[dfcom['codeInsee'] == codeInsee].id.to_list()
    print(list_coms)
    dftest = dfevt
    dftest['is_create'] = dftest['creation'].apply(lambda x: not set(list_coms).isdisjoint(x))
    df1 = dftest[dftest['is_create'] == True]
    dftest['is_supp'] = dftest['suppression'].apply(lambda x: not set(list_coms).isdisjoint(x))
    df2 = dftest[dftest['is_supp'] == True]
    df3 = pd.concat([df1,df2])
    return df3[~df3.index.duplicated(keep='first')].sort_values(by=['nom'])

In [23]:
def getEvtByNom(nom):
    list_coms = dfcom[dfcom['nom'] == nom].id.to_list()
    print(list_coms)
    dftest = dfevt
    dftest['is_create'] = dftest['creation'].apply(lambda x: not set(list_coms).isdisjoint(x))
    df1 = dftest[dftest['is_create'] == True]
    dftest['is_supp'] = dftest['suppression'].apply(lambda x: not set(list_coms).isdisjoint(x))
    df2 = dftest[dftest['is_supp'] == True]
    df3 = pd.concat([df1,df2])
    return df3[~df3.index.duplicated(keep='first')].sort_values(by=['nom'])

In [24]:
getEvtByCodeInsee('27032')

['d171728d-5c4a-48a7-83c2-f62e8db01f99', 'a0dc41c9-8026-4382-9f5c-5b4075ab90e8', 'bfb5f656-4440-4561-bb49-e2136b9b7803']


,creation,suppression,nom,id,type,creationObj,suppressionObj,is_create,is_supp
4469,"[4c09a74f-4b9d-4c2c-9e03-5574c979aa97, 0ce410a...",[],1943-01-01,904dcf14-b895-4686-a5c5-7304c6f63cae,evenement,"[{'codeInsee': '32019', 'nom': 'Auterive', 'no...",[],True,False
563,"[d171728d-5c4a-48a7-83c2-f62e8db01f99, bfb5f65...","[a1ee0f67-b657-418d-822b-7eaab68a89eb, 826d072...",2016-01-01,40ae74a8-6433-4dfa-a388-c98b629954d5,evenement,"[{'codeInsee': '27032', 'nom': 'Chambois', 'no...","[{'codeInsee': '27634', 'nom': 'Thomer-la-Sôgn...",True,True
5060,[],"[bfb5f656-4440-4561-bb49-e2136b9b7803, 9bd0400...",2020-12-31,4d54af6e-0a1a-4b4f-970b-3aa95d51e21e,evenement,[],[],False,True


In [25]:
getEvtByNom('Avrilly')

['a0dc41c9-8026-4382-9f5c-5b4075ab90e8', '0c3cf932-0ae5-42b8-93ab-e1a0e13af1c1', '832adfcd-9b9f-40d1-a3fb-17b5e58196d1', 'bfb5f656-4440-4561-bb49-e2136b9b7803']


,creation,suppression,nom,id,type,creationObj,suppressionObj,is_create,is_supp
4469,"[4c09a74f-4b9d-4c2c-9e03-5574c979aa97, 0ce410a...",[],1943-01-01,904dcf14-b895-4686-a5c5-7304c6f63cae,evenement,"[{'codeInsee': '32019', 'nom': 'Auterive', 'no...",[],True,False
563,"[d171728d-5c4a-48a7-83c2-f62e8db01f99, bfb5f65...","[a1ee0f67-b657-418d-822b-7eaab68a89eb, 826d072...",2016-01-01,40ae74a8-6433-4dfa-a388-c98b629954d5,evenement,"[{'codeInsee': '27032', 'nom': 'Chambois', 'no...","[{'codeInsee': '27634', 'nom': 'Thomer-la-Sôgn...",True,True
5060,[],"[bfb5f656-4440-4561-bb49-e2136b9b7803, 9bd0400...",2020-12-31,4d54af6e-0a1a-4b4f-970b-3aa95d51e21e,evenement,[],[],False,True


Analyse nombre création / suppression par événement

In [26]:
dfevt['nb_creation'] = dfevt['creation'].apply(lambda x: len(x))
dfevt['nb_suppression'] = dfevt['suppression'].apply(lambda x: len(x))

Combien d'événement ont concerné plus d'une commune en création et plus d'une commune en suppression :

In [30]:
dfevt[(dfevt['nb_creation'] > 1) & (dfevt['nb_suppression'] > 1)].shape[0]

1798

Combien d'événement ont impliqués plus de communes créées que de communes supprimées :

In [31]:
dfevt[(dfevt['nb_creation'] > 1) & (dfevt['nb_suppression'] > 1) & (dfevt['nb_creation'] >= dfevt['nb_suppression']+1)].shape[0]

657

On à toujours au max qu'une seule création en plus que de suppression :

In [32]:
dfevt[(dfevt['nb_creation'] > 1) & (dfevt['nb_suppression'] > 1) & (dfevt['nb_creation'] >= dfevt['nb_suppression']+2)].shape[0]

0

On va rechercher les événements qui ont créé plus d'une commune COM :

In [48]:
dftest = dfevt[(dfevt['nb_creation'] > 1) & (dfevt['nb_suppression'] > 1) & (dfevt['nb_creation'] >= dfevt['nb_suppression']+1)]

In [51]:
arr = []
for index, row in dftest.iterrows():
    nb_com_in = 0
    nb_coma_in = 0
    nb_comd_in = 0
    nb_com_out = 0
    nb_coma_out = 0
    nb_comd_out = 0
    for com in row['creationObj']:
        if(com['type'] == 'COM'):
            nb_com_out = nb_com_out + 1
        if(com['type'] == 'COMD'):
            nb_comd_out = nb_comd_out + 1
        if(com['type'] == 'COMA'):
            nb_coma_out = nb_coma_out + 1
    for com in row['suppressionObj']:
        if(com['type'] == 'COM'):
            nb_com_in = nb_com_in + 1
        if(com['type'] == 'COMD'):
            nb_comd_in = nb_comd_in + 1
        if(com['type'] == 'COMA'):
            nb_coma_in = nb_coma_in + 1
    if(nb_com_out > 1):
        arr.append(row)
        print('---------')
        print('Evenement '+row['id']+' : ')
        print('Suppression : '+str(nb_com_in)+' communes, '+str(nb_comd_in)+' communes déléguées, '+str(nb_coma_in)+' communes associées')
        print('Creation : '+str(nb_com_out)+' communes, '+str(nb_comd_out)+' communes déléguées, '+str(nb_coma_out)+' communes associées')

dfout = pd.DataFrame(arr)

---------
Evenement 637f4e23-fb2d-4f95-8bd6-36c78c699b3b : 
Suppression : 3 communes, 0 communes déléguées, 0 communes associées
Creation : 4 communes, 0 communes déléguées, 0 communes associées
---------
Evenement 38377343-88a6-458b-8142-162370e38638 : 
Suppression : 2 communes, 0 communes déléguées, 0 communes associées
Creation : 3 communes, 0 communes déléguées, 0 communes associées
---------
Evenement 2c4e2074-25ce-40a1-b7d3-39e28a238982 : 
Suppression : 4 communes, 0 communes déléguées, 0 communes associées
Creation : 5 communes, 0 communes déléguées, 0 communes associées
---------
Evenement 662cfa72-c4e3-49d1-94bb-7ee067c3a726 : 
Suppression : 8 communes, 0 communes déléguées, 0 communes associées
Creation : 9 communes, 0 communes déléguées, 0 communes associées
---------
Evenement 2cc22e41-441b-4897-ad76-22fc1cd17415 : 
Suppression : 2 communes, 0 communes déléguées, 0 communes associées
Creation : 3 communes, 0 communes déléguées, 0 communes associées
---------
Evenement dff90

In [61]:
for index, row in dfout.iterrows():
    print('-------')
    mystr = ''
    for com in row['suppressionObj']:
        mystr = mystr+com['nom']+' ('+com['codeInsee']+' - '+com['type']+') ; '
    
    mystr = mystr + '\ndeviennent\n'
                                          
    for com in row['creationObj']:
        mystr = mystr+com['nom']+' ('+com['codeInsee']+' - '+com['type']+') ; '
    print(mystr)

-------
Vaulnaveys-le-Haut (38529 - COM) ; Séchilienne (38478 - COM) ; Saint-Martin-d'Uriage (38422 - COM) ; 
deviennent
Chamrousse (38567 - COM) ; Vaulnaveys-le-Haut (38529 - COM) ; Séchilienne (38478 - COM) ; Saint-Martin-d'Uriage (38422 - COM) ; 
-------
Bures-sur-Yvette (91122 - COM) ; Orsay (91471 - COM) ; 
deviennent
Bures-sur-Yvette (91122 - COM) ; Les Ulis (91692 - COM) ; Orsay (91471 - COM) ; 
-------
Saint-Jean-Lespinasse (46271 - COM) ; Saint-Vincent-du-Pendit (46295 - COM) ; Saint-Médard-de-Presque (46281 - COM) ; Saint-Céré (46251 - COM) ; 
deviennent
Saint-Médard-de-Presque (46281 - COM) ; Saint-Vincent-du-Pendit (46295 - COM) ; Saint-Jean-Lespinasse (46271 - COM) ; Saint-Jean-Lagineste (46339 - COM) ; Saint-Céré (46251 - COM) ; 
-------
Porte-Joie (27471 - COM) ; Poses (27474 - COM) ; Tournedos-sur-Seine (27651 - COM) ; Léry (27365 - COM) ; Saint-Pierre-du-Vauvray (27598 - COM) ; Incarville (27351 - COM) ; Saint-Étienne-du-Vauvray (27537 - COM) ; Le Vaudreuil (27528 - CO